In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import networkx as nx
from matplotlib import pyplot as plt
from rdflib import Graph
from rdflib.plugins.sparql import prepareQuery


#### Load Graph

In [ ]:
g = Graph() # the oxigraph stuff makes queries run faster

g.parse('KnowledgeGraphs/knowledge_graph.ttl', format="turtle")
g.parse('KnowledgeGraphs/ontology_graph.ttl', format="turtle")

In [ ]:
def format_large_number(num):
    num = float(num)
    if num < 1000:
        return str(num)
    elif num < 1000000:
        return "{:.2f}K".format(num / 1000)
    elif num < 1000000000:
        return "{:.2f}M".format(num / 1000000)
    elif num < 1000000000000:
        return "{:.2f}B".format(num / 1000000000)
    else:
        return "{:.2f}T".format(num / 1000000000000)


In [ ]:
##### Query artists with more followers on Twitter than on Youtube

In [196]:
queries ="""
prefix dbo:     <http://dbpedia.org/ontology/> 
PREFIX rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs:    <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wdp: <https://www.wikidata.org/wiki/Property:>
PREFIX wd: <https://www.wikidata.org/wiki/>

SELECT ?artistLabel ?youtubeFollowers ?twitterFollowers
WHERE {
    ?artist rdf:type wd:Q639669;
            rdfs:label ?artistLabel .
    ?artist schema:hasYoutubeFollowers ?youtubeFollowers .
    ?artist schema:hasTwitterFollowers ?twitterFollowers .
    FILTER(xsd:integer(?youtubeFollowers) < xsd:integer(?twitterFollowers))
}
"""

query = prepareQuery(queries)
result = g.query(query)

for row in result:
    print('Artist:', row.artistLabel,'has more followers on Twitter than on Youtube. Youtube Followers:', format_large_number(row.youtubeFollowers), 'Twitter Followers:', format_large_number(row.twitterFollowers))
    print('*'*50)

Artist: Def Leppard has more followers on Twitter than on Youtube. Youtube Followers: 913.00K Twitter Followers: 957.09K
**************************************************
Artist: Shakira has more followers on Twitter than on Youtube. Youtube Followers: 45.40M Twitter Followers: 53.76M
**************************************************
Artist: Paul McCartney has more followers on Twitter than on Youtube. Youtube Followers: 1.27M Twitter Followers: 4.28M
**************************************************
Artist: Barbra Streisand has more followers on Twitter than on Youtube. Youtube Followers: 493.00K Twitter Followers: 798.97K
**************************************************
Artist: Britney Spears has more followers on Twitter than on Youtube. Youtube Followers: 10.00M Twitter Followers: 57.98M
**************************************************
Artist: Neil Diamond has more followers on Twitter than on Youtube. Youtube Followers: 343.00K Twitter Followers: 867.02K
*******************

In [27]:
#### Find how many artists have more followers on Twitter than on Youtube ( along with the average number of followers on each platform)

In [28]:
queries = """
PREFIX schema: <https://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT (COUNT(DISTINCT ?artistYT) AS ?YoutubeMore)
       (COUNT(DISTINCT ?artistTW) AS ?TwitterMore)
       (AVG(xsd:integer(?youtubeFollowers)) AS ?AvgYoutubeFollowers)
       (AVG(xsd:integer(?twitterFollowers)) AS ?AvgTwitterFollowers)
WHERE {
  {
    SELECT ?artist
           (MAX(?youtubeFollowers) AS ?youtubeFollowers)
           (MAX(?twitterFollowers) AS ?twitterFollowers)
           (IF(MAX(xsd:integer(?youtubeFollowers)) > MAX(xsd:integer(?twitterFollowers)), ?artist, "") AS ?artistYT)
           (IF(MAX(xsd:integer(?twitterFollowers)) > MAX(xsd:integer(?youtubeFollowers)), ?artist, "") AS ?artistTW)
    WHERE {
      ?artist schema:hasYoutubeFollowers ?youtubeFollowers ;
              schema:hasTwitterFollowers ?twitterFollowers .
    }
    GROUP BY ?artist
  }
}

"""


query = prepareQuery(queries)
result = g.query(query)

for row in result:
    print('Youtube Followers:', row.YoutubeMore, 'Twitter Followers:', row.TwitterMore, 'Avg Youtube Followers:', format_large_number(row.AvgYoutubeFollowers), 'Avg Twitter Followers:', format_large_number(row.AvgTwitterFollowers))

Youtube Followers: 44 Twitter Followers: 25 Avg Youtube Followers: 12.33M Avg Twitter Followers: 14.43M


On average artists have more followers on Twitter than on Youtube (somewhat surprising), since Youtube is a more popular platform for music. However, twitter has been scrutinized for having a lot of fake accounts, so this could be a reason for the difference.

It is not surprising that most artists have more followers on Youtube than on Twitter.


In [ ]:
## This query finds avg lifetime sales of artists, grouping them per country.

In [198]:


queries = """
PREFIX schema: <https://schema.org/>
PREFIX wdp: <https://www.wikidata.org/wiki/Property:>
PREFIX wd: <https://www.wikidata.org/wiki/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?country (COUNT(?artist) AS ?numArtists) (AVG(xsd:double(?sales)) AS ?avgSales)
WHERE {
    ?artist wdp:P27 ?countryID .
    ?artist schema:hasLifeTimeSales ?sales .
    ?countryID rdfs:label ?countryLabel .
    BIND(str(?countryLabel) AS ?country)
}
GROUP BY ?country
ORDER BY DESC(?avgSales)

"""


query = prepareQuery(queries)
result = g.query(query)

for row in result:
    print('Country: ', row.country, 'Avg TCU (millions):', round(float(row.avgSales),2), 'Number of Artists:', row.numArtists)
    print('*'*50)

Country:  Barbados Avg TCU (millions): 335.3 Number of Artists: 1
**************************************************
Country:  CanadaUnited States Avg TCU (millions): 277.9 Number of Artists: 1
**************************************************
Country:  Canada Avg TCU (millions): 180.83 Number of Artists: 5
**************************************************
Country:  Trinidad and Tobago Avg TCU (millions): 156.3 Number of Artists: 1
**************************************************
Country:  United Kingdom Avg TCU (millions): 109.59 Number of Artists: 20
**************************************************
Country:  United States Avg TCU (millions): 98.98 Number of Artists: 79
**************************************************
Country:  United Kingdom United States Avg TCU (millions): 94.4 Number of Artists: 1
**************************************************
Country:  Ireland Avg TCU (millions): 83.25 Number of Artists: 2
**************************************************
Country:  A

It seems our data is not very useful in this case, too few artists


In [205]:
queries = """
PREFIX schema: <https://schema.org/>
PREFIX wdp: <https://www.wikidata.org/wiki/Property:>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX wd: <https://www.wikidata.org/wiki/>

SELECT ?SalesInterval (AVG(?numberOfChildren) AS ?AvgNumberOfChildren)
WHERE {
    ?artist schema:hasLifeTimeSales ?sales ;
          wdp:P1552 ?family ;
          rdfs:label ?artistLabel .
  ?family a wd:Q8436;
          wdp:P1971 ?numberOfChildrenRaw .
          
          
  BIND(xsd:integer(?numberOfChildrenRaw) AS ?numberOfChildren)
    BIND(
        IF(?sales <= 50, "0-50 million",
          IF(?sales <= 100, "51-100 million",
            IF(?sales <= 150, "101-150 million",
              IF(?sales <= 200, "151-200 million",
                "Above 200 million")
            )
          )
        ) AS ?SalesInterval
  )
}
GROUP BY ?SalesInterval
ORDER BY ?SalesInterval

"""


query = prepareQuery(queries)
result = g.query(query)

for row in result:
    print('Sales Interval:', row.SalesInterval, 'Avg Number of Children:', round(float(row.AvgNumberOfChildren),2))
    print('*'*50)

Sales Interval: 0-50 million Avg Number of Children: 2.71
**************************************************
Sales Interval: 101-150 million Avg Number of Children: 2.33
**************************************************
Sales Interval: 151-200 million Avg Number of Children: 2.67
**************************************************
Sales Interval: 51-100 million Avg Number of Children: 2.55
**************************************************
Sales Interval: Above 200 million Avg Number of Children: 2.43
**************************************************


It's quite clear that there is no substantial difference between number of children and their lifetime sales.


#### Simple Query - Find the most successful artists in terms of number of top 10 songs

In [206]:
queries = """
PREFIX schema: <https://schema.org/>
PREFIX wdp: <https://www.wikidata.org/wiki/Property:>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?artistLabel (COUNT(?song) AS ?songCount)
WHERE {
  ?song wdp:P175 ?artist .  # ?song is performed by ?artist
  ?artist rdfs:label ?artistLabel .
}
GROUP BY ?artist
ORDER BY DESC(?songCount)
LIMIT 10
"""


query = prepareQuery(queries)
result = g.query(query)

for row in result:
    print('', row.artistLabel, '|Number of Songs:', row.songCount)
    print('*'*50)

 Drake |Number of Songs: 52
**************************************************
 Madonna |Number of Songs: 35
**************************************************
 The Beatles |Number of Songs: 34
**************************************************
 Michael Jackson |Number of Songs: 28
**************************************************
 Rihanna |Number of Songs: 27
**************************************************
 Taylor Swift |Number of Songs: 26
**************************************************
 Elvis Presley |Number of Songs: 25
**************************************************
 Stevie Wonder |Number of Songs: 25
**************************************************
 Elton John |Number of Songs: 24
**************************************************
 The Rolling Stones |Number of Songs: 23
**************************************************


Get the top 10 songs of each artist and find the most successful labels in terms of number of 10 songs.

In [208]:
queries = """
PREFIX schema: <https://schema.org/>
PREFIX wdp: <https://www.wikidata.org/wiki/Property:>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?labelName (COUNT(?song) AS ?top10Count)
WHERE {
  ?song wdp:P175 ?artist .                # ?song is performed by ?artist.
  ?artist wdp:P264 ?label .               # ?artist is signed to ?label.
  ?label rdfs:label ?labelName .          # Get the label's name.
  ?song wdp:P175 ?artist .                # ?song is performed by ?artist.
  ?song wdp:P1352 ?ranking .              # ?song has a ?ranking.
  ?ranking schema:value ?rankValue .      # Get the ranking value.
  FILTER(?rankValue <= 10)                # Consider only top 10 ranked songs.
}
GROUP BY ?labelName
ORDER BY DESC(?top10Count)

LIMIT 20
"""


query = prepareQuery(queries)
result = g.query(query)


for row in result:    
    print('Label Name:', row.labelName, 'Number of Top 10 Songs:', row.top10Count)
    print('*'*50)

Label Name:  'Roc Nation' Number of Top 10 Songs: 299
**************************************************
Label Name:  'Westbury Road Entertainment' Number of Top 10 Songs: 299
**************************************************
Label Name: 'Def Jam Recordings' Number of Top 10 Songs: 299
**************************************************
Label Name:  'Butterfly MC Records' Number of Top 10 Songs: 286
**************************************************
Label Name:  'Epic Records' Number of Top 10 Songs: 286
**************************************************
Label Name:  'Monarc Entertainment' Number of Top 10 Songs: 286
**************************************************
Label Name:  'Virgin Music' Number of Top 10 Songs: 286
**************************************************
Label Name: 'Sony Music' Number of Top 10 Songs: 286
**************************************************
Label Name: 'LaFace Records' Number of Top 10 Songs: 246
**************************************************
Label

This gives us an idea of the most successful labels in terms of top 10 songs. The approach is somehwat flawed, as an artist can have multiple labels, and a song can have multiple artists. This does not take into account if the artist achieved the top 10 ranking while signed to the label.

In [ ]:
#### Relationship Between Public Image and Social Media Presence

#### Get Public Image of Artist and followers on Twitter and Youtube (see if there is a correlation)

In [210]:
queries = """
PREFIX schema: <https://schema.org/>
PREFIX wdp: <https://www.wikidata.org/wiki/Property:>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wd: <https://www.wikidata.org/wiki/>

SELECT DISTINCT ?artistName ((?twitterFollowers+ ?youtubeFollowers)AS ?totalFollowers) ?publicImageScore ?publicImage
WHERE {
  ?artist a wd:Q639669 .
  ?artist rdfs:label ?artistName .
  ?artist schema:hasTwitterFollowers ?twitterFollowers .
  ?artist schema:hasYoutubeFollowers ?youtubeFollowers .
  
  ?artist wdp:P1552 ?characteristic .
  ?characteristic a wd:Q6738447 ;
                  wdp:P444 ?publicImageScore .
  
  BIND(
    IF(?publicImageScore > 10, "Positive", 
      IF(?publicImageScore > 0, "Mixed", "Negative")
    ) AS ?publicImage
  )
}
ORDER BY DESC(?publicImageScore)
LIMIT 20

"""
query = prepareQuery(queries)
result = g.query(query)


for row in result:    
    # print('Artist:', row.artistName,'|Public Image Score:',row.publicImageScore,'|Public Image:',row.publicImage ,'|Twitter Followers:', format_large_number(row.twitterFollowers), '|Youtube Followers:', format_large_number(row.youtubeFollowers))
    print('Artist:', row.artistName,'|Public Image Score:',row.publicImageScore,'|Public Image:',row.publicImage ,'|Total Followers:', format_large_number(row.totalFollowers))
    print('*'*50)

Artist: Mariah Carey |Public Image Score: 111.6931 |Public Image: Positive |Total Followers: 32.51M
**************************************************
Artist: Shakira |Public Image Score: 65.51752 |Public Image: Positive |Total Followers: 99.16M
**************************************************
Artist: Elvis Presley |Public Image Score: 62.77248 |Public Image: Positive |Total Followers: 418.52K
**************************************************
Artist: Michael Jackson |Public Image Score: 59.98027 |Public Image: Positive |Total Followers: 32.05M
**************************************************
Artist: Ariana Grande |Public Image Score: 59.10966 |Public Image: Positive |Total Followers: 129.49M
**************************************************
Artist: Beyonce |Public Image Score: 58.63214 |Public Image: Positive |Total Followers: 42.47M
**************************************************
Artist: Led Zeppelin |Public Image Score: 57.33621 |Public Image: Positive |Total Followers: 4.10

#### Get Public Image of Artist and First Post Date (see if nostalgia has an effect on public image)

In [212]:
queries = """
PREFIX schema: <https://schema.org/>
PREFIX wdp: <https://www.wikidata.org/wiki/Property:>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wd: <https://www.wikidata.org/wiki/>

SELECT ?artistName ?publicImage ?publicImageScore (MIN(?datePosted) AS ?firstPostDate)
WHERE {
  ?artist a wd:Q639669 .
  ?artist rdfs:label ?artistName .
  ?song wdp:P175 ?artist ;
        wdp:P1352 ?ranking .
  ?ranking schema:datePosted ?datePosted .
  
  ?artist wdp:P1552 ?characteristic .
    ?characteristic a wd:Q6738447 ;
                    wdp:P444 ?publicImageScore .
  
   BIND(
    IF(?publicImageScore > 10, "Positive", 
      IF(?publicImageScore > 0, "Mixed", "Negative")
    ) AS ?publicImage
  )

}
GROUP BY ?artistName
ORDER BY DESC(?publicImageScore)
LIMIT 20
"""
query = prepareQuery(queries)
result = g.query(query)


for row in result:    
    print('Artist:', row.artistName,'|Public Image Score:',row.publicImageScore,'|Public Image:',row.publicImage ,'|First Post Date:', row.firstPostDate)
    print('*'*50)

Artist: Mariah Carey |Public Image Score: 111.6931 |Public Image: Positive |First Post Date: 1990-07-21
**************************************************
Artist: Frank Sinatra |Public Image Score: 80.54346 |Public Image: Positive |First Post Date: 1966-06-04
**************************************************
Artist: Alicia Keys |Public Image Score: 74.83484 |Public Image: Positive |First Post Date: 2001-08-04
**************************************************
Artist: Shakira |Public Image Score: 65.51752 |Public Image: Positive |First Post Date: 2001-12-15
**************************************************
Artist: Elvis Presley |Public Image Score: 62.77248 |Public Image: Positive |First Post Date: 1958-08-04
**************************************************
Artist: Whitney Houston |Public Image Score: 61.34449 |Public Image: Positive |First Post Date: 1985-07-06
**************************************************
Artist: Michael Jackson |Public Image Score: 59.98027 |Public Image: Po